In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import requests 
import json
from shutil import which
import time
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from lxml import etree

In [2]:
def extract(page_source, start_sep, end_sep):
    source=page_source

    result=[]
    tmp=source.split(start_sep)
    for par in tmp:
        if end_sep in par:
            result.append(par.split(end_sep)[0])
    return result

def extract2(string, start, end):
    result = re.search('%s(.*)%s' % (start, end), string).group(1)
    return result

In [3]:
def empty_item():
    global product_details
    product_details = {
        'name': None,
        'sku': None,
        'gtin': None,
        'color': None,
        'finish': None,
        'price': None,
        'product_description': None,
        'technical_description': None,
        'display_dimensions': None,
        'length': None,
        'width': None,
        'height': None,
        'depth': None,
        'diameter': None,
        'weight': None,
        'arm_height': None,
        'seat_height': None,
        'table_clearance_height': None,
        'maximum_dimensions': None,
        'hero_image': None,
        'variant_image': None,
        'images': None,
        'files': None,
        'installations': None,
        'designer': None,
        'collection': None,
        'categories': None,
        'available_new': None,
        'made_in': None,
        'voltage': None,
        'bulb_type': None,
        'bulb_text': None,
        'canopy_dimensions': None,
        'ul_listed': None,
        'certifications': None,
        'warranty_&_care': None,
        'position': None,
        'notes': None,
        'url':  None
    }
    return product_details




In [97]:
chrome_options = Options()
headless = False

if headless:
    chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")


prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

# PLease replace your chrome driver path
global driver
driver = webdriver.Chrome(executable_path=r"C:\Users\kchai\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe", options=chrome_options)

# driver.get("https://www.piancausa.com/collections/all")

items = []
exit = True
i = 1
while exit:
    driver.get(f"https://www.piancausa.com/collections/all?page={i}")
    print(i)
    print('*\n' * 3)
    i+=1
    product_url = driver.find_elements(By.XPATH, "//div[@class = 'grid-uniform grid-link__container']/div/div/a")
    if len(product_url)<1:
        exit = False
        
    for product in product_url:
        empty_item()

        href = product.get_attribute("href")
        driver.execute_script("window.open('" +href +"');")
        print(href)
        product_details['url'] = href
    #     switch to new window
        driver.switch_to.window(driver.window_handles[1])
        
        response = requests.get(href)
        page_source = response.text

        data = extract(page_source, 'window.ShopifyAnalytics.lib.track("Viewed Product",', ');' )
        data = json.loads(data[-1])
        
        product_details['sku'] = data['sku']
        product_details['price'] = data['price']
        product_details['categories'] = data['category']
        product_details['productId'] = data['productId']
        product_details['variantId'] = data['variantId']
        product_details['variant'] = data['variant']
        product_details['brand'] = data['brand']
        
        time.sleep(1)
        product_details['name'] = driver.find_element(By.XPATH, "//h1[@class = 'product-single__title']").text

        images = []
        images_path = driver.find_elements(By.XPATH, "//ul[@class = 'product-single__thumbnails grid-uniform']/li/a")
        for img in images_path:
            images.append(img.get_attribute('href'))

        product_details['images'] = '\n'.join(images)

        product_details['hero_image'] = images[0]
        product_details['variant_image'] = images[0]

        product_info = []


        desc_info = driver.find_elements(By.XPATH, "//div[@class = 'product-description rte']//p/span")

        for info in desc_info:
            if (len(info.text) >3) and ('Dimensions' not in info.text) and ('Download Technical Sheet' not in info.text) \
                and ('Designed by:' not in info.text):
                product_info.append(info.text)
            if 'Designed by:' in info.text:
                product_details['designer'] = info.text.split(':')[-1][1:]

        product_details['product_description'] = '\n'.join(product_info)

        if len(product_details['product_description'])< 10:
            product_info = []
            desc_info = driver.find_elements(By.XPATH, "//div[@class = 'product-description rte']/p")

            for info in desc_info:
                if (len(info.text) >3) and ('Dimensions' not in info.text) and ('Download Technical Sheet' not in info.text)\
                    and ('Designed by:' not in info.text):
                    product_info.append(info.text)
                if 'Designed by:' in info.text:
                    product_details['designer'] = info.text.split(':')[-1][1:]

            product_details['product_description'] = '\n'.join(product_info)

        if len(product_details['product_description'])< 10:
            product_info = []
            desc_info = driver.find_elements(By.XPATH, "//div[@class = 'bt_bb_text']/p")

            for info in desc_info:
                if (len(info.text) >3) and ('Dimensions' not in info.text) and ('Download Technical Sheet' not in info.text)\
                    and ('Designed by:' not in info.text):
                    product_info.append(info.text)
                if 'Designed by:' in info.text:
                    product_details['designer'] = info.text.split(':')[-1][1:]

                if 'Dimensions' in info.text:
                    product_details['technical_description'] = info.text



            product_details['product_description'] = '\n'.join(product_info)

        if (product_details['technical_description'] is None) or (len(product_details['technical_description']) < 2):
            technical_description = []
            tech_desc = driver.find_elements(By.XPATH, "//div[@class = 'product-description rte']//p//strong")
            for dim in tech_desc:
                if ('Designed by:' not in dim.text) and ('Design by:' not in dim.text) and ('Download' not in dim.text)\
                and ('Technical Sheet' not in dim.text):
                    technical_description.append(dim.text)
            product_details['technical_description'] = ' '.join(technical_description)

        if len(product_details['technical_description']) < 2 :
            print('*'*10)
            tech_desc = driver.find_element(By.XPATH, "//div[@class = 'bt_bb_column_content_inner']/div/strong")
            product_details['technical_description'] = tech_desc.text

        try:
            files_path = driver.find_element(By.XPATH, "//div[@class = 'product-description rte']/p//a")
            product_details['files'] = "Technical Sheet: " + files_path.get_attribute('href')
        except:
            try:
                files_path = driver.find_element(By.XPATH, "//div[@class = 'bt_bb_column_content_inner']//p/a")
                product_details['files'] = "Technical Sheet: " + files_path.get_attribute('href')
            except:
                product_details['files'] = None

#         price_path = driver.find_element(By.XPATH, "//span[@id= 'ProductPrice']")
#         product_details['price'] = price_path.text.split('$')[-1]

        quantity_path = driver.find_element(By.XPATH, "//div[@id= 'variant-inventory'][1]/span")
        product_details['quantity'] = quantity_path.text.split(':')[-1][1:]

        avaliable_path = driver.find_element(By.XPATH, "//div[@id= 'variant-inventory'][3]/span")
        product_details['available_new'] = avaliable_path.text

        option_path = driver.find_element(By.XPATH, "//div[@class= 'selector-wrapper']/select/option")
        if len(option_path.text) > 1:
            product_details['option(title)'] = option_path.text

#         sku_path = driver.find_element(By.XPATH, "//select[@class= 'product-single__variants']/option")
#         product_details['sku'] = sku_path.get_attribute('data-sku')



        driver.switch_to.window(driver.window_handles[1])

        print(product_details)
        items.append(product_details)
        driver.close()

        #back to main window
        driver.switch_to.window(driver.window_handles[0])

C:\Users\kchai\AppData\Local\Temp\ipykernel_195132\3692568084.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\kchai\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe", options=chrome_options)


1
*
*
*

https://www.piancausa.com/collections/all/products/copy-of-abaco-low-marble-glass-coffee-table-35-calacatta-matt-marble-glass-titanium-hg
{'name': 'ABACO LOW ROUND COFFE TABLE 35" - CALACATTA MARBLE GLASS/TITANIUM', 'sku': '110-10000-0250-00', 'gtin': None, 'color': None, 'finish': None, 'price': '2192.00', 'product_description': 'The simplicity of its design conceals interesting and well-gauged geometric and constructive choices that explore and exploit the theme of the circle and symmetry.\nThe base is composed of three slender metal legs, connected by a spoked structure that also supports the top, which can be raised for use as a handy tray, separated from the structure by a simple gesture.\nThe combinations of finishes for the top and base can be playful or elegant, just as the combinations of multiple tables from the Abaco series can be varied and original.', 'technical_description': 'Dimensions: Diam. 35.00" x H.7.00"', 'display_dimensions': None, 'length': None, 'width'

{'name': 'ABACO LOW ROUND COFFEE TABLE 35" - HIGH GLOSS BRONZE', 'sku': '110-10000-0271-00', 'gtin': None, 'color': None, 'finish': None, 'price': '2192.00', 'product_description': 'The simplicity of its design conceals interesting and well-gauged geometric and constructive choices that explore and exploit the theme of the circle and symmetry.\nThe base is composed of three slender metal legs, connected by a spoked structure that also supports the top, which can be raised for use as a handy tray, separated from the structure by a simple gesture.\nThe combinations of finishes for the top and base can be playful or elegant, just as the combinations of multiple tables from the Abaco series can be varied and original.', 'technical_description': 'Dimensions: Diam .35.00" x H.7.00"', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimen

{'name': 'ABACO LOW ROUND COFFEE TABLE 35" - WHITE MATT LACQUER', 'sku': '110-10000-0019-00', 'gtin': None, 'color': None, 'finish': None, 'price': '1015.00', 'product_description': 'The simplicity of its design conceals interesting and well-gauged geometric and constructive choices that explore and exploit the theme of the circle and symmetry.\nThe base is composed of three slender metal legs, connected by a spoked structure that also supports the top, which can be raised for use as a handy tray, separated from the structure by a simple gesture.\nThe combinations of finishes for the top and base can be playful or elegant, just as the combinations of multiple tables from the Abaco series can be varied and original.', 'technical_description': 'Dimensions: Diam.35.00" x H.7.00"', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimen

{'name': 'ABACO MED.ROUND COFFEE TABLE 30" - HIGH GLOSSY TITANIUM', 'sku': '110-10000-0260-00', 'gtin': None, 'color': None, 'finish': None, 'price': '2091.00', 'product_description': 'The simplicity of its design conceals interesting and well-gauged geometric and constructive choices that explore and exploit the theme of the circle and symmetry.\nThe base is composed of three slender metal legs, connected by a spoked structure that also supports the top, which can be raised for use as a handy tray, separated from the structure by a simple gesture.\nThe combinations of finishes for the top and base can be playful or elegant, just as the combinations of multiple tables from the Abaco series can be varied and original.', 'technical_description': 'Dimensions: Diam. 30" x H 12"', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensi

...

{'name': 'PIUMOTTO QUEEN SIZE BED 75" - LIGHT BEIGE AQUARIUS 01 FABRIC', 'sku': '140-10000-0261-00', 'gtin': None, 'color': None, 'finish': None, 'price': '4151.00', 'product_description': 'The refined simplicity of the Embrace bed echoes the warmth of an embrace, a retreat to a safe haven. The dominating feature of the bed, the curved headboard, cocoons the main body of the bed in an inviting, material embrace, offering protective, reassuring shoulders. ', 'technical_description': 'Dimensions: W.67.32"/74.80" x H.43.30" x D.92.51', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0289/5944/0975/products/PIUMO4_1024x1024.png?v=1666792810', 'variant_image': 'https://cdn.shopify.com/s/files/1/0289/5944/0975/products/PIUMO4_1024x1024.png?v=1666792810', 'images': 'http

{'name': 'RADA QUEEN SIZE BED 81" - BEIGE ARTEPELLE 25 ECO-LEATHER/HG BRONZE', 'sku': '140-10000-0283-00', 'gtin': None, 'color': None, 'finish': None, 'price': '6026.00', 'product_description': 'Understated style, inviting contours, softly combined. Rada is a bed with a flowing, intimate and delicate profile. The headboard curves backwards gently, inviting total relaxation, yielding to its form. The bed appears to derive from a single pen-stroke, its curves flowing seamlessly into metal feet, in a supremely delicate yet natural contrast of materials.', 'technical_description': 'Dimensions: W.65.74" x H.36.22" x D.90.15" ', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0289/5944/0975/products/RADA3_1024x1024.png?v=1666795331', 'variant_image': 'https://cdn.shopi

{'name': 'SERVOQUADRO END TABLE 19" - WHITE MATT/TITANIUM', 'sku': '110-10000-0018-00', 'gtin': None, 'color': None, 'finish': None, 'price': '396.00', 'product_description': "Elegant, contemporary design for a coffee table that is not afraid to embrance a retro feel. \nThe square version is called Sevoquadro and the round version Servogiro. The frame is designed to wrap around the sofa's seat and make a valuable complement to any chill-out zone.\nDetails: White Matt Lacquer with Black Titanium Structure.", 'technical_description': 'Details: White Matt Lacquer with Black Titanium Structure. Overall Dimensions: W18.9" x  H.18.9" x D.21.2"', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0289/5944/0975/products/SERVOCUADRO_1024x1024.png?v=1666716366', 'variant_imag

{'name': 'SPAZIO WALL UNIT COMP. 94" - SMOKED OAK/BEIGE SETA MATT LACQUER', 'sku': '130-10000-0143-00', 'gtin': None, 'color': None, 'finish': None, 'price': '5061.00', 'product_description': 'Simplifying everyday movements, creating contours that command order and harmony: Spazio is the very essence of functionality.\nWith close attention to details and a love of textures, it offers a range of modules, sizes and finishes unrivaled in their variety and flexibility. Spazio is ideal for every room in the home and has something to satisfy the tastes and requirements of everyone.', 'technical_description': 'Dimensions:  W.94.49" x H.47.64" x D.17.72"', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0289/5944/0975/products/13010000014300_1024x1024.jpg?v=1662060239', '

{'name': 'SPAZIO WALL UNIT COMP.94" - BURNT OAK/GREY ROCCIA MATT LACQUER', 'sku': '130-10000-0142-00', 'gtin': None, 'color': None, 'finish': None, 'price': '5061.00', 'product_description': 'Simplifying everyday movements, creating contours that command order and harmony: Spazio is the very essence of functionality.\nWith close attention to details and a love of textures, it offers a range of modules, sizes and finishes unrivaled in their variety and flexibility. Spazio is ideal for every room in the home and has something to satisfy the tastes and requirements of everyone.', 'technical_description': 'Dimensions:  W.94.49" x H.47.64" x D.17.72"', 'display_dimensions': None, 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0289/5944/0975/products/13010000014200_2_1024x1024.jpg?v=1650550959', 

In [98]:
len(items)

172

In [110]:
import pandas as pd
import numpy as np

df = pd.DataFrame(items, )
df


,name,sku,gtin,color,finish,price,product_description,technical_description,display_dimensions,length,...,warranty_&_care,position,notes,url,productId,variantId,variant,brand,quantity,option(title)
0,"ABACO LOW ROUND COFFE TABLE 35"" - CALACATTA MA...",110-10000-0250-00,None,None,None,2192.00,The simplicity of its design conceals interest...,"Dimensions: Diam. 35.00"" x H.7.00""",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6640486711375,39678856396879,Titanium Hg + Calacatta Matt Marble Glass,PIANCA USA,6,Titanium Hg + Calacatta Matt Marble Glass
1,"ABACO LOW ROUND COFFEE TABLE 35"" - ASH GREY CE...",110-10000-0241-00,None,None,None,1015.00,The simplicity of its design conceals interest...,"Dimensions: W. 35.00"" x H.7.00""",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6639706341455,39676856631375,Cenere Matt Lacquered Top + Lavagna Matt Lacqu...,PIANCA USA,6,Cenere Matt Lacquered Top + Lavagna Matt Lacqu...
2,"ABACO LOW ROUND COFFEE TABLE 35"" - BLACK SAHAR...",110-10000-0253-00,None,None,None,2192.00,The simplicity of its design conceals interest...,"Dimensions: Diam.35.00"" x H.7.00""",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6638983544911,39674828521551,Bronze Hg + Sahara Matt Marble Glass,PIANCA USA,7,Bronze Hg + Sahara Matt Marble Glass
3,"ABACO LOW ROUND COFFEE TABLE 35"" - HIGH GLOSS ...",110-10000-0271-00,None,None,None,2192.00,The simplicity of its design conceals interest...,"Dimensions: Diam .35.00"" x H.7.00""",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6638966440015,39674787758159,Bronze Glossy,PIANCA USA,1,Bronze Glossy
4,"ABACO LOW ROUND COFFEE TABLE 35"" - HIGH GLOSS ...",110-10000-0272-00,None,None,None,2192.00,The simplicity of its design conceals interest...,"Dimensions: Diam. 35.00"" x H.7.00""",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6638967488591,39674791297103,Titanium Glossy,PIANCA USA,1,Titanium Glossy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,"SPAZIO WALL UNIT COMP.126"" - NATURAL OAK/BEIGE...",130-10000-0159-00,None,None,None,8300.00,Spazio Tv Wall Unit Composition. Two lateral P...,"Overall Dimensions: W.125.98"" x H.70.86"" x D.1...",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6690806890575,39849802498127,Natural Oak (Rovere Naturale) and Beige (Seta)...,PIANCA USA,1,Natural Oak (Rovere Naturale) and Beige (Seta)...
168,"SPAZIO WALL UNIT COMP.126"" - WALNUT/FOG GREY N...",130-10000-0157-00,None,None,None,8300.00,Spazio Tv Wall Unit Composition. Two lateral P...,"Overall Dimensions: W.125.98"" x H.70.86"" x D.1...",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6690806431823,39849800138831,Walnut (Canaletto) and Fog Grey (Nebbia) matt ...,PIANCA USA,,Walnut (Canaletto) and Fog Grey (Nebbia) matt ...
169,"SPAZIO WALL UNIT COMP.94"" - BURNT OAK/GREY ROC...",130-10000-0142-00,None,None,None,5061.00,"Simplifying everyday movements, creating conto...","Dimensions: W.94.49"" x H.47.64"" x D.17.72""",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6700602818639,39874564620367,Rovere Borgogna e Roccia matt Lacquer,PIANCA USA,,Rovere Borgogna e Roccia matt Lacquer
170,"SPAZIO/DOMINO SM DESK+DRESSER 90"" - BEIGE SETA...",140-10000-0254-00,None,None,None,3007.00,"A desk, a workstation, as well as a table. Dom...","Overall Dimensions: W.49.12""/90.5""x H.31.73""x ...",None,None,...,None,None,None,https://www.piancausa.com/collections/all/prod...,6607847030863,39588889460815,None,PIANCA USA,,NaN


In [111]:
len(df['new_name'].unique())

KeyError: 'new_name'

In [112]:
for i in range(len(df)):
    temp = df['technical_description'][i].split('Dimensions:')[-1].split('x')
    for dim in temp:
        if ('Diam.' in dim):
            df['diameter'][i] = dim.split('Diam.')[-1]
        elif ('Diam .' in dim):
            df['diameter'][i] = dim.split('Diam .')[-1]
        elif 'W.' in dim:
            df['width'][i] = dim.split('W.')[-1]
        elif 'H.' in dim:
            df['height'][i] = dim.split('H.')[-1]
        elif 'H ' in dim:
            df['height'][i] = dim.split('H ')[-1]
        elif 'D.' in dim:
            df['depth'][i] = dim.split('D.')[-1]

C:\Users\kchai\AppData\Local\Temp\ipykernel_195132\1352925675.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diameter'][i] = dim.split('Diam.')[-1]
C:\Users\kchai\AppData\Local\Temp\ipykernel_195132\1352925675.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['height'][i] = dim.split('H.')[-1]
C:\Users\kchai\AppData\Local\Temp\ipykernel_195132\1352925675.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['width'][i] 

In [113]:
df['name']

0      ABACO LOW ROUND COFFE TABLE 35" - CALACATTA MA...
1      ABACO LOW ROUND COFFEE TABLE 35" - ASH GREY CE...
2      ABACO LOW ROUND COFFEE TABLE 35" - BLACK SAHAR...
3      ABACO LOW ROUND COFFEE TABLE 35" - HIGH GLOSS ...
4      ABACO LOW ROUND COFFEE TABLE 35" - HIGH GLOSS ...
                             ...                        
167    SPAZIO WALL UNIT COMP.126" - NATURAL OAK/BEIGE...
168    SPAZIO WALL UNIT COMP.126" - WALNUT/FOG GREY N...
169    SPAZIO WALL UNIT COMP.94" - BURNT OAK/GREY ROC...
170    SPAZIO/DOMINO SM DESK+DRESSER 90" - BEIGE SETA...
171    SPAZIO/DOMINO SM DESK+DRESSER 90" - WHITE MATT...
Name: name, Length: 172, dtype: object

In [114]:
df['new_name'] = None
for i in range(len(df)):
    df['new_name'][i] = df['name'][i].split('-')[0]
    

C:\Users\kchai\AppData\Local\Temp\ipykernel_195132\2622097750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_name'][i] = df['name'][i].split('-')[0]


In [115]:
df['new_name']

0        ABACO LOW ROUND COFFE TABLE 35" 
1       ABACO LOW ROUND COFFEE TABLE 35" 
2       ABACO LOW ROUND COFFEE TABLE 35" 
3       ABACO LOW ROUND COFFEE TABLE 35" 
4       ABACO LOW ROUND COFFEE TABLE 35" 
                      ...                
167           SPAZIO WALL UNIT COMP.126" 
168           SPAZIO WALL UNIT COMP.126" 
169            SPAZIO WALL UNIT COMP.94" 
170    SPAZIO/DOMINO SM DESK+DRESSER 90" 
171    SPAZIO/DOMINO SM DESK+DRESSER 90" 
Name: new_name, Length: 172, dtype: object

In [116]:
df.to_excel('piancausa.xlsx', index = False)

In [95]:
df['diameter'].isna().sum()

142

In [137]:
df = pd.read_excel(r'C:\Users\kchai\Downloads\piancausa (1).xlsx')
df

,name,new_name,sku,gtin,color,finish,price,product_description,technical_description,display_dimensions,...,certifications,warranty_&_care,position,notes,url,productId,variantId,variant,quantity,option(title)
0,"ABACO LOW ROUND COFFE TABLE 35"" - CALACATTA MA...","ABACO LOW ROUND COFFEE TABLE 35""",110-10000-0250-00,NaN,NaN,NaN,2192.0,The simplicity of its design conceals interest...,"Dimensions: Diam. 35.00"" x H.7.00""",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6640486711375,39678856396879,Titanium Hg + Calacatta Matt Marble Glass,6.0,Titanium Hg + Calacatta Matt Marble Glass
1,"ABACO LOW ROUND COFFEE TABLE 35"" - ASH GREY CE...","ABACO LOW ROUND COFFEE TABLE 35""",110-10000-0241-00,NaN,NaN,NaN,1015.0,The simplicity of its design conceals interest...,"Dimensions: W. 35.00"" x H.7.00""",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6639706341455,39676856631375,Cenere Matt Lacquered Top + Lavagna Matt Lacqu...,6.0,Cenere Matt Lacquered Top + Lavagna Matt Lacqu...
2,"ABACO LOW ROUND COFFEE TABLE 35"" - BLACK SAHAR...","ABACO LOW ROUND COFFEE TABLE 35""",110-10000-0253-00,NaN,NaN,NaN,2192.0,The simplicity of its design conceals interest...,"Dimensions: Diam.35.00"" x H.7.00""",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6638983544911,39674828521551,Bronze Hg + Sahara Matt Marble Glass,7.0,Bronze Hg + Sahara Matt Marble Glass
3,"ABACO LOW ROUND COFFEE TABLE 35"" - HIGH GLOSS ...","ABACO LOW ROUND COFFEE TABLE 35""",110-10000-0271-00,NaN,NaN,NaN,2192.0,The simplicity of its design conceals interest...,"Dimensions: Diam .35.00"" x H.7.00""",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6638966440015,39674787758159,Bronze Glossy,1.0,Bronze Glossy
4,"ABACO LOW ROUND COFFEE TABLE 35"" - HIGH GLOSS ...","ABACO LOW ROUND COFFEE TABLE 35""",110-10000-0272-00,NaN,NaN,NaN,2192.0,The simplicity of its design conceals interest...,"Dimensions: Diam. 35.00"" x H.7.00""",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6638967488591,39674791297103,Titanium Glossy,1.0,Titanium Glossy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,"SPAZIO WALL UNIT COMP.126"" - NATURAL OAK/BEIGE...","SPAZIO WALL UNIT COMP.126""",130-10000-0159-00,NaN,NaN,NaN,8300.0,Spazio Tv Wall Unit Composition. Two lateral P...,"Overall Dimensions: W.125.98"" x H.70.86"" x D.1...",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6690806890575,39849802498127,Natural Oak (Rovere Naturale) and Beige (Seta)...,1.0,Natural Oak (Rovere Naturale) and Beige (Seta)...
168,"SPAZIO WALL UNIT COMP.126"" - WALNUT/FOG GREY N...","SPAZIO WALL UNIT COMP.126""",130-10000-0157-00,NaN,NaN,NaN,8300.0,Spazio Tv Wall Unit Composition. Two lateral P...,"Overall Dimensions: W.125.98"" x H.70.86"" x D.1...",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6690806431823,39849800138831,Walnut (Canaletto) and Fog Grey (Nebbia) matt ...,NaN,Walnut (Canaletto) and Fog Grey (Nebbia) matt ...
169,"SPAZIO WALL UNIT COMP.94"" - BURNT OAK/GREY ROC...","SPAZIO WALL UNIT COMP.94""",130-10000-0142-00,NaN,NaN,NaN,5061.0,"Simplifying everyday movements, creating conto...","Dimensions: W.94.49"" x H.47.64"" x D.17.72""",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6700602818639,39874564620367,Rovere Borgogna e Roccia matt Lacquer,NaN,Rovere Borgogna e Roccia matt Lacquer
170,"SPAZIO/DOMINO SM DESK+DRESSER 90"" - BEIGE SETA...","SPAZIO/DOMINO SM DESK+DRESSER 90""",140-10000-0254-00,NaN,NaN,NaN,3007.0,"A desk, a workstation, as well as a table. Dom...","Overall Dimensions: W.49.12""/90.5""x H.31.73""x ...",NaN,...,NaN,NaN,NaN,NaN,https://www.piancausa.com/collections/all/prod...,6607847030863,39588889460815,NaN,NaN,NaN


In [138]:
dim_col = ['diameter', 'width', 'height', 'depth']

In [139]:
df['new_sku'] = None
df['dimesnsions'] = None
df['technical_description_upt'] = None
for i in range(len(df)):
    df['new_sku'][i] = df['variant_image'][i].split('/')[-1].split('_')[0]
    
    if 'overall dimensions:' in df['technical_description'][i].lower():
        dim = df['technical_description'][i].split('Overall Dimensions:')[-1].strip()
        info = df['technical_description'][i].split('Overall Dimensions:')[0].strip()
        
    elif 'dimensions:' in df['technical_description'][i].lower():
        dim = df['technical_description'][i].split('Dimensions:')[-1].strip()
        info = df['technical_description'][i].split('Dimensions:')[0].strip()
    
    if len(info) >5:
            df['technical_description_upt'][i] = info
    else:
        df['technical_description_upt'][i] = None
        
    if len(dim) >3:
            df['dimesnsions'][i] = dim
    else:
        df['dimesnsions'][i] = None
            
    for col in dim_col:
        try:
            df[col][i] = float(df[col][i].replace('"',''))
        except:
            pass

C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\1757412682.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_sku'][i] = df['variant_image'][i].split('/')[-1].split('_')[0]
C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\1757412682.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['technical_description_upt'][i] = None
C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\1757412682.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [140]:
names = list(df['new_name'].unique())
names

['ABACO LOW ROUND COFFEE TABLE 35" ',
 'ABACO MED.ROUND COFFEE TABLE 30" ',
 'ABACO MEDIUM COFFEE TABLE 30" ',
 'ABACO MEDIUM ROUND COFFEE TABLE 30" ',
 'ABACO TALL ROUND COFFEE TABLE 24"',
 'ABACO TALL ROUND COFFEE TABLE 24" ',
 'ALA HANGING DESK W/GROMET 47" ',
 'ALA WALL DESK W/GROMET 47" ',
 'ALL',
 'ALL IN ARMCHAIR LOW BACKREST 42" ',
 'ALL IN SOFA LOW BACKREST 120" ',
 'ALUNNA ARMCHAIR 22" ',
 'ALUNNA SIDE CHAIR 22" ',
 'ATLANTE 2 DRWRS NIGHTST. 24" ',
 'ATLANTE 2 DRWRS NIGHTST. 28" ',
 'ATLANTE 2 DRWRS NIGHTST. 29" ',
 'ATLANTE 3 DRWRS DRESSER 60" ',
 'ATLANTE 3 DRWRS DRESSER MATT 60" ',
 'ATLANTE 3 DRWS DRESSER 60" ',
 'ATLANTE DRESSER 60" ',
 'BOSTON CAPITONNE\' 114" ',
 'BOSTON CAPITONNE\' SOFA 114" ',
 'BOSTON CAPITONNE\' SOFA 98" ',
 'BOSTON PLAIN SECTIONAL SOFA 112" ',
 'CHLOE VANITY DESK 46" ',
 'CONTRALTO SIDE TABLE 15"',
 'CONTRALTO SIDE TABLE 15" ',
 'CORNICE SIDEBOARD 82" ',
 'DEDALO 1 DRWR NIGHTST STACKABLE./WALNUT AND TUTANIUM PLINTH ',
 'DEDALO 2 DRWRS DRESSER 67" 

In [141]:
df['hero_image_upd'] = None
df_upt = pd.DataFrame()
for name in names:
    df_temp = df[df['new_name'] == name]

    df_temp['hero_image_upd'].iloc[0] = df_temp['hero_image'].iloc[0]    

    df_upt = df_upt.append(df_temp, ignore_index=True)

C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\2079940361.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['hero_image_upd'].iloc[0] = df_temp['hero_image'].iloc[0]
C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\2079940361.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_upt = df_upt.append(df_temp, ignore_index=True)


In [142]:
df_upt['dimesnsions'].unique()

array(['Diam. 35.00" x H.7.00"', 'W. 35.00" x H.7.00"',
       'Diam.35.00" x H.7.00"', 'Diam .35.00" x H.7.00"',
       'Diam. 29.92" x H.12"', 'Diam. 30" x H.12"', 'Diam. 30" x H 12"',
       'Diam. 29.92" x H 12"', 'Diam.23.62" x H.18"', 'Diam. 24" x H.18"',
       'Diam.24" x H.18"', 'W.47.24" x H.27.55" x D.24"',
       'W.42" x H.29" x P.30"', 'W.120.08" x H.29.13" x P.37.79"',
       'W.22" x H.30" x D.20.5"', 'W.21.7" x H.30" x D.20.5"',
       'W.24.80" x H.20.08" x D.17.72"', 'W.28.74" x H.20.08" x D.17.72"',
       'W.60.24" x H.29.92" x D.17.72"',
       'W.114.17" x H.26.38" x D.39.37"', 'W.114" x H.26.4" x D.39.4"',
       'W.98.42" x H.26.38" x D.39.37', 'W.98.4" x H.26.3" x D.39.3"',
       'W.112.2" x H.26.4" x D.82.7"', 'W.46.06" x H.36.22" x D.21.65"',
       'Diam.15" x H.26"', 'W.82.67" xH.27.55" x D.17.71"',
       'W.17.32" x H.9.84"', 'W.67.32" x H.29.53" x D.21.65"',
       'W.67.32" H.29.53" x D.21.65"', 'Diam.17"x H.18.5"',
       'W.17.32" x H.19"', 'Diam .2

In [143]:
df_upt.to_excel('piancausa_upd.xlsx', index = False)

In [70]:
df[df['new_name'] == 'ATLANTE 2 DRWRS NIGHTST. 29" ']['hero_image'][1:] = None

C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\3634820979.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df[df['new_name'] == 'ATLANTE 2 DRWRS NIGHTST. 29" ']['hero_image'][1:] = None
C:\Users\kchai\AppData\Local\Temp\ipykernel_9428\3634820979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['new_name'] == 'ATLANTE 2 DRWRS NIGHTST. 29" ']['hero_image'][1:] = None
